In [ ]:
db_path = './data/upload/chinook.db'


['albums', 'sqlite_sequence', 'artists', 'customers', 'employees', 'genres', 'invoices', 'invoice_items', 'media_types', 'playlists', 'playlist_track', 'tracks', 'sqlite_stat1']


In [7]:
import sqlite3
import json

def extract_all_metadata(db_path, output_file):
    """
    Extract metadata for all tables in a SQLite database
    and store it in a JSON file.

    Parameters:
        db_path (str): Path to the SQLite database file.
        output_file (str): Path to the JSON file to save metadata.
    """
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Get all table names
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
        tables = [row[0] for row in cursor.fetchall()]

        all_metadata = []

        # Loop through each table and extract metadata
        for table in tables:
            cursor.execute(f"PRAGMA table_info({table});")
            columns = cursor.fetchall()

            metadata = []
            for col in columns:
                metadata.append({
                    "cid": col[0],            # Column ID
                    "name": col[1],           # Column name
                    "type": col[2],           # Data type
                    "notnull": bool(col[3]),  # Whether column is NOT NULL
                    "primary_key": bool(col[5]) # Is part of primary key
                })

            # Add table metadata
            all_metadata.append({
                "table_name": table,
                "columns": metadata
            })

        # Close connection
        conn.close()

        # Save to JSON file
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(all_metadata, f, indent=4)

        print(f"Metadata for all tables saved to {output_file}")

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")


In [8]:

output='./data/feature/output.json'
meta = extract_all_metadata(db_path, output)
print(meta)

Metadata for all tables saved to ./data/feature/output.json
None


In [9]:
import json

def extract_table_names(metadata_file):
    """
    Extract all table names from a metadata JSON file.
    """
    try:
        # Load JSON data from file
        with open(metadata_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Extract table names
        table_names = [entry["table_name"] for entry in data if "table_name" in entry]
        return table_names

    except (json.JSONDecodeError, FileNotFoundError) as e:
        print(f"Error reading metadata file: {e}")
        return []


In [10]:
tabl = extract_table_names(output)
print(tabl)

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'sqlite_sequence', 'sqlite_stat1', 'tracks']


In [ ]:
import json

def get_table_metadata_from_file(metadata_file, table_name):
    """
    Extract metadata for a specific table from a metadata JSON file.
    """
    try:
        # Load JSON data from file
        with open(metadata_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Search for the table
        for entry in data:
            if entry.get("table_name") == table_name:
                return entry

        return None

    except (json.JSONDecodeError, FileNotFoundError) as e:
        print(f"Error reading metadata file: {e}")
        return None


In [13]:
details = get_table_metadata_from_file(output, 'albums')
print(details)

{'table_name': 'albums', 'columns': [{'cid': 0, 'name': 'AlbumId', 'type': 'INTEGER', 'notnull': True, 'primary_key': True}, {'cid': 1, 'name': 'Title', 'type': 'NVARCHAR(160)', 'notnull': True, 'primary_key': False}, {'cid': 2, 'name': 'ArtistId', 'type': 'INTEGER', 'notnull': True, 'primary_key': False}]}
